# Bibliotecas

In [100]:
# Imports
import numpy as np 
import pandas as pd 
import sys
import json
import os
import re
import unicodedata
import string
import random

from sklearn.feature_extraction import text      
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.svm import NuSVC
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('rslp')

from unidecode import unidecode
import matplotlib.pyplot as plt
from fractions import Fraction

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


# Carregando os datasets

In [101]:
# Import dataset artistas
artists = pd.read_csv('https://raw.githubusercontent.com/digonfernan/brazilian-lyrics-classifier/main/brazilian-lyrics-classifier/artists-data.csv')

# Visualização inicial dos dados
display(artists.shape)
display(artists.head())
display(artists.tail())

(4168, 5)

,Artist,Genres,Songs,Popularity,Link
0,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/
1,Chiclete com Banana,Axé,268.0,3.8,/chiclete-com-banana/
2,Banda Eva,Axé; Romântico; Reggae,215.0,2.3,/banda-eva/
3,É O Tchan,Axé,129.0,1.6,/e-o-tchan/
4,Claudia Leitte,Pop; Axé; Romântico,167.0,1.5,/claudia-leitte/


,Artist,Genres,Songs,Popularity,Link
4163,Miriam Makeba,World Music; Black Music; Blues,17.0,0.0,/miriam-makeba/
4164,Freddie Aguilar,World Music,61.0,0.0,/freddie-aguilar/
4165,Amadou & Mariam,World Music,14.0,0.0,/amadou-mariam/
4166,Magic System,World Music; Gospel/Religioso,16.0,0.0,/magic-system/
4167,Johnny Clegg,World Music; Gospel/Religioso,34.0,0.0,/clegg-johnny/


In [102]:
# Import dataset letras
lyrics = pd.read_csv('https://media.githubusercontent.com/media/digonfernan/brazilian-lyrics-classifier/main/brazilian-lyrics-classifier/lyrics-data.csv')

# Visualização inicial dos dados
display(lyrics.shape)
display(lyrics.head())
display(lyrics.tail())

(379931, 5)

,ALink,SName,SLink,Lyric,language
0,/ivete-sangalo/,Arerê,/ivete-sangalo/arere.html,"Tudo o que eu quero nessa vida,\nToda vida, é\...",pt
1,/ivete-sangalo/,Se Eu Não Te Amasse Tanto Assim,/ivete-sangalo/se-eu-nao-te-amasse-tanto-assim...,Meu coração\nSem direção\nVoando só por voar\n...,pt
2,/ivete-sangalo/,Céu da Boca,/ivete-sangalo/chupa-toda.html,É de babaixá!\nÉ de balacubaca!\nÉ de babaixá!...,pt
3,/ivete-sangalo/,Quando A Chuva Passar,/ivete-sangalo/quando-a-chuva-passar.html,Quando a chuva passar\n\nPra quê falar\nSe voc...,pt
4,/ivete-sangalo/,Sorte Grande,/ivete-sangalo/sorte-grande.html,A minha sorte grande foi você cair do céu\nMin...,pt


,ALink,SName,SLink,Lyric,language
379926,/clegg-johnny/,The Waiting,/clegg-johnny/the-waiting.html,Chorus\nHere we stand waiting on the plain\nDa...,en
379927,/clegg-johnny/,Too Early For The Sky,/clegg-johnny/too-early-for-the-sky.html,I nearly disappeared into the mouth of a croco...,en
379928,/clegg-johnny/,Warsaw 1943 (I Never Betrayed The Revolution),/clegg-johnny/warsaw-1943-i-never-betrayed-the...,"Amambuka, amambuka azothengisa izwe lakithi, i...",en
379929,/clegg-johnny/,When The System Has Fallen,/clegg-johnny/when-the-system-has-fallen.html,Sweat in the heat for days on end\nwaiting for...,en
379930,/clegg-johnny/,Woman Be My Country,/clegg-johnny/woman-be-my-country.html,Here we stand on the edge of the day\nFaces me...,en


# Tratamento dos dados

Como esse notebook irá levar em consideração apenas músicas em português, irei filtrar as músicas em português e depois unir os datasets.

In [103]:
# Cria novo dataset somente com músicas em português
lyrics_pt = lyrics[lyrics['language']=='pt']

# Visualiza os dados
display(lyrics_pt.shape)
display(lyrics_pt.head())

(157393, 5)

,ALink,SName,SLink,Lyric,language
0,/ivete-sangalo/,Arerê,/ivete-sangalo/arere.html,"Tudo o que eu quero nessa vida,\nToda vida, é\...",pt
1,/ivete-sangalo/,Se Eu Não Te Amasse Tanto Assim,/ivete-sangalo/se-eu-nao-te-amasse-tanto-assim...,Meu coração\nSem direção\nVoando só por voar\n...,pt
2,/ivete-sangalo/,Céu da Boca,/ivete-sangalo/chupa-toda.html,É de babaixá!\nÉ de balacubaca!\nÉ de babaixá!...,pt
3,/ivete-sangalo/,Quando A Chuva Passar,/ivete-sangalo/quando-a-chuva-passar.html,Quando a chuva passar\n\nPra quê falar\nSe voc...,pt
4,/ivete-sangalo/,Sorte Grande,/ivete-sangalo/sorte-grande.html,A minha sorte grande foi você cair do céu\nMin...,pt


In [104]:
# Inner join para unir os datasets e remover os valores faltantes
musics_pt = pd.merge(artists, lyrics_pt, how='inner', left_on='Link', right_on='ALink')
musics_pt = musics_pt.dropna()

# Visualiza os dados
display(musics_pt.shape)
display(musics_pt.head())

(156940, 10)

,Artist,Genres,Songs,Popularity,Link,ALink,SName,SLink,Lyric,language
0,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/,/ivete-sangalo/,Arerê,/ivete-sangalo/arere.html,"Tudo o que eu quero nessa vida,\nToda vida, é\...",pt
1,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/,/ivete-sangalo/,Se Eu Não Te Amasse Tanto Assim,/ivete-sangalo/se-eu-nao-te-amasse-tanto-assim...,Meu coração\nSem direção\nVoando só por voar\n...,pt
2,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/,/ivete-sangalo/,Céu da Boca,/ivete-sangalo/chupa-toda.html,É de babaixá!\nÉ de balacubaca!\nÉ de babaixá!...,pt
3,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/,/ivete-sangalo/,Quando A Chuva Passar,/ivete-sangalo/quando-a-chuva-passar.html,Quando a chuva passar\n\nPra quê falar\nSe voc...,pt
4,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/,/ivete-sangalo/,Sorte Grande,/ivete-sangalo/sorte-grande.html,A minha sorte grande foi você cair do céu\nMin...,pt


In [105]:
# Devido ao tamanho do dataset, o mesmo será limitado aleatoriamente apenas a 3% da sua capacidade
musics_pt = musics_pt.sample(frac=0.03)

# Visualiza os dados
display(musics_pt.shape)
display(musics_pt.head())

(4708, 10)

,Artist,Genres,Songs,Popularity,Link,ALink,SName,SLink,Lyric,language
135652,Matogrosso e Mathias,Sertanejo; Romântico,292.0,3.7,/matogrosso-mathias/,/matogrosso-mathias/,Poesia Não Se Vende,/matogrosso-mathias/poesia-nao-se-vende.html,Poesia não se vende\nEntão não fale em dinheir...,pt
31368,MC Sabrina,Funk Carioca; Pagode; Hip Hop,47.0,0.0,/mc-sabrina/,/mc-sabrina/,Diretoria,/mc-sabrina/diretoria.html,O natural joga no balão\nPra amenizar o nosso ...,pt
136347,Trio Parada Dura,Sertanejo,369.0,2.5,/trio-parada-dura/,/trio-parada-dura/,Me Espere Essa Noite,/trio-parada-dura/me-espere-essa-noite.html,Estou lhe telefonando\nPra dizer que acabei de...,pt
75376,Erasmo Carlos,Rock; Jovem Guarda; Romântico,343.0,5.6,/erasmo-carlos/,/erasmo-carlos/,A Grande Mágoa,/erasmo-carlos/a-grande-magoa.html,A grande mágoa\nQue nunca tem fim\nVive gritan...,pt
98681,Katinguelê,Samba; Pagode,138.0,1.1,/katinguele/,/katinguele/,Engraçadinha,/katinguele/engracadinha.html,"Linda de viver, que gracinha minha tentação\nC...",pt


In [106]:
# Drop das colunas que estão repetindo ou não são relevantes
musics_pt.drop(['Link', 'ALink', 'SLink', 'language'], axis=1, inplace=True)

# Visualiza os dados
musics_pt.sample(10)

,Artist,Genres,Songs,Popularity,SName,Lyric
38503,Gabriela Rocha,Gospel/Religioso,77.0,7.2,Nossa Canção,Somos estrangeiros\nCansados de andar\nProcura...
22103,Gatinha Manhosa,Forró,203.0,0.3,Só Dessa Vez,"E corre, entra e bate a porta\nJura que não va..."
121091,Daniel,Sertanejo; Romântico,392.0,5.4,A Paixão Acaba Sempre Assim,Eu saí\nFeito um louco amando por aí\nMe engan...
96821,Bezerra da Silva,Pagode; Samba,295.0,1.8,A Giria É A Cultura do Povo,Toda hora tem gíria no asfalto e no morro\npor...
102538,Skank,Pop/Rock; Rock; Pop,141.0,3.9,Trancoso,Um colar de contas daqui vou te levar\nUm cola...
16381,Luamarte,Pop; MPB; Folk,13.0,0.0,Espumas ao Vento,Sei que aí dentro ainda mora um pedacinho de m...
87986,Fafá de Belém,MPB; Romântico,284.0,1.0,Gostoso,Gostoso é se saber que é sempre bem\nChegado e...
47409,Amanda Ferrari,Gospel/Religioso,103.0,1.7,Uma Hora Dessa,"Uma Hora Dessa\n\nUma hora dessa, uma hora des..."
59685,Denise Cerqueira,Gospel/Religioso; Romântico; Pop,68.0,0.0,Meu Clamor,Luar diz porque chorou?\nQuando me encontrou l...
34443,Padre Zezinho,Gospel/Religioso,713.0,20.6,Outra Canção Por Maria,"Estou pensando em ti, Maria de Jesus\nEstou pe..."


In [107]:
# Transforma as todas palavras em minúscula e remove os acentos 
musics_pt['Lyric'] = musics_pt['Lyric'].str.lower().apply(lambda x: unidecode(x))

# Remove pontuação
musics_pt['Lyric'] = musics_pt['Lyric'].str.replace('[{}]'.format(string.punctuation), ' ')

# Remove números
musics_pt['Lyric'] = musics_pt['Lyric'].str.replace('[{}]'.format(string.digits), '')

# Remove caracteres especiais
musics_pt['Lyric'] = musics_pt['Lyric'].str.replace ('[/<>()|\+\-\$%&#@\'\"]+', ' ', regex=True).copy()

# Substitui nova linha (\n) por espaço vazio
musics_pt['Lyric'] = musics_pt['Lyric'].str.replace('\n', ' ')

# Stop words
stop = stopwords.words('portuguese')
stop2 = list()
for word in stop:
  stop2.append(unidecode(word))
  
stemmer = nltk.stem.RSLPStemmer()

musics_pt['Lyric'] = musics_pt['Lyric'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop2)]))

# Visualiza os dados
musics_pt.sample(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  


,Artist,Genres,Songs,Popularity,SName,Lyric
16641,Calcinha Preta,Forró; Romântico,316.0,9.3,Hello,bb f gm eb f bb morrendo saudade tanto tempo v...
143347,Cleber e Cauan,Sertanejo,87.0,2.0,Sfav,entendi gente bem corpo corpo coracao quer sab...
37533,Gerson Rufino,Gospel/Religioso,244.0,8.1,Gratidão,pois passarinho pode cantar avoa fica triste p...
28292,Bryan Tiller,Funk; Rap; Hip Hop,39.0,0.0,Amsterdã,gosta cash yeah remexe yeah banda mexe yeah ra...
20145,Elba Ramalho,Forró; MPB,265.0,3.4,Leão do Norte,coracao folclore nordestino mateus bastiao boi...
49674,Mara Dalila,Gospel/Religioso,191.0,1.3,Senhor Por Ti Viverei,i ti confiei senhor ti depositei dor ti descan...
118401,Bruno e Marrone,Romântico; Sertanejo,387.0,8.3,Telefone Mudo,quero risque nome agenda esqueca telefone ligu...
92360,Filipe Catto,MPB,74.0,0.0,Do Fundo do Coração,noite princesa caida mim lago peito secreta so...
46996,Andréa Fontes,Gospel/Religioso,198.0,1.8,Não Há Limites,pensas tudo terminado saida pra pensas sonhos ...
152989,Roger & Rogério,Sertanejo,66.0,0.0,Não Vou Superar,hoje acordei olhei pro lado vi travesseiro per...


# Construção do modelo

In [108]:
# Divisão do dataset para treino e teste
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(musics_pt, test_size=0.2, random_state=25)

print(f'Números de exemplos de treino: {train_data.shape[0]}')
print(f'Números de exemplos de teste {test_data.shape[0]}')

Números de exemplos de treino: 3766
Números de exemplos de teste 942


In [109]:
# Cria vocabulário
cv = CountVectorizer(strip_accents='ascii', lowercase=True, stop_words=stop2)
cv.fit(train_data['Lyric'].values)

# Bag of words
bow = cv.transform(train_data['Lyric'].values) 
print(bow.shape[0], 'amostras x',bow.shape[1],'palavras no vocabulário')

3766 amostras x 24512 palavras no vocabulário


# Treinamento do modelo

In [110]:
# Cria o machine learning classifier object
models = {'Logistic Regression':LogisticRegression(max_iter=250), 'Linear SVC':LinearSVC(max_iter=5000),'Decision Tree': DecisionTreeClassifier(), 'Gradient Descent': SGDClassifier()}
for the_model in models.keys():
    print ('Training', the_model)
    models[the_model].fit(bow.toarray(), train_data['Artist'])

Training Logistic Regression


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Training Linear SVC


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


Training Decision Tree
Training Gradient Descent


In [111]:
# Converte lyrics para bag of word
test_bow = cv.transform(test_data['Lyric'].values) 

In [112]:
# Predict
accuracy = {}
pg = test_data[['Artist', 'SName']]
for the_model in models.keys():
    print('Evaluating',the_model)
    pred_artist= models[the_model].predict(test_bow.toarray())
    pg[the_model]=pred_artist
    accuracy[the_model]=accuracy_score(test_data['Artist'],pred_artist)
print ("---Accuracy Scores---")
print(accuracy)

Evaluating Logistic Regression


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


Evaluating Linear SVC


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


Evaluating Decision Tree


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


Evaluating Gradient Descent
---Accuracy Scores---
{'Logistic Regression': 0.029723991507430998, 'Linear SVC': 0.025477707006369428, 'Decision Tree': 0.020169851380042462, 'Gradient Descent': 0.005307855626326964}


# Input

In [ ]:
# Input
user_input = input()

# Transforma em bag of words
user_bow = cv.transform([(user_input)]) 

# Roda os modelos treinados
for the_model in models.keys():
    print(the_model, models[the_model].predict(user_bow))